In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


##Run this in Google Colab

In [2]:
from getpass import getpass
import os

GITHUB_REPO = "Suryasnata1404/Finance-Insight"
print("Repo:", GITHUB_REPO)

token = getpass("Paste your GitHub PAT (input hidden): ")
clone_url = f"https://{token}@github.com/{GITHUB_REPO}.git"

# clone into /content/repo
!rm -rf /content/repo
!git clone {clone_url} /content/repo

# show files
import os
print("Repo files:", os.listdir("/content/repo")[:40])


Repo: Suryasnata1404/Finance-Insight
Paste your GitHub PAT (input hidden): ··········
Cloning into '/content/repo'...
remote: Enumerating objects: 186, done.
remote: Counting objects: 100% (186/186), done.
remote: Compressing objects: 100% (113/113), done.
remote: Total 186 (delta 88), reused 159 (delta 61), pack-reused 0 (from 0)
Receiving objects: 100% (186/186), 1.60 MiB | 7.36 MiB/s, done.
Resolving deltas: 100% (88/88), done.
Repo files: ['scripts', '01_test_ner.ipynb', '.gitignore', 'requirements.txt', 'README.md', '.git', 'notebooks', 'data']


In [6]:
# unzip dataset into repo data/processed/
!unzip -q /content/drive/MyDrive/Finance-Insight/data/processed/ner_auto_splits.zip -d /content/repo/data/processed/
!ls -la /content/repo/data/processed/ner_auto_splits | sed -n '1,200p'


unzip:  cannot find or open /content/drive/MyDrive/Finance-Insight/data/processed/ner_auto_splits.zip, /content/drive/MyDrive/Finance-Insight/data/processed/ner_auto_splits.zip.zip or /content/drive/MyDrive/Finance-Insight/data/processed/ner_auto_splits.zip.ZIP.
total 6332
drwxr-xr-x 2 root root    4096 Nov  6 08:35 .
drwxr-xr-x 3 root root    4096 Nov  6 08:35 ..
-rw-r--r-- 1 root root     388 Nov  6 08:35 label_map.json
-rw-r--r-- 1 root root  442844 Nov  6 08:35 test.jsonl
-rw-r--r-- 1 root root 5504055 Nov  6 08:35 train.jsonl
-rw-r--r-- 1 root root  519794 Nov  6 08:35 validation.jsonl


In [ ]:
!ls -lh /content/drive/MyDrive/Finance-Insight/data/processed/ner_auto_splits


total 6.2M
-rw------- 1 root root  411 Oct 31 11:41 label_map.json
-rw------- 1 root root 434K Oct 31 11:41 test.jsonl
-rw------- 1 root root 5.3M Oct 31 11:41 train.jsonl
-rw------- 1 root root 509K Oct 31 11:41 validation.jsonl


In [ ]:
# Step 1: Install required libraries
!pip install -q transformers datasets evaluate seqeval accelerate huggingface_hub

# Step 2: Check versions and GPU availability
import transformers, datasets, torch

print("transformers:", transformers.__version__)
print("datasets:", datasets.__version__)
print("torch:", torch.__version__, "| CUDA available:", torch.cuda.is_available())


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.3 MB/s eta 0:00:00
transformers: 4.57.1
datasets: 4.0.0
torch: 2.8.0+cu126 | CUDA available: True


In [7]:
# ✅ Step 1: Set paths
REPO_DIR = "/content/repo"   # where your GitHub repo was cloned
TRAIN_FILE = f"{REPO_DIR}/scripts/7_train_quick_ner.py"   # 👈 change if your filename differs
DRIVE_OUT = "/content/drive/MyDrive/Finance-Insight/ner_results"

import os, sys

# ✅ Step 2: Verify the repo contents
print("📁 Repo files:")
for f in os.listdir(REPO_DIR):
    print(" -", f)

# ✅ Step 3: Check training file
if not os.path.exists(TRAIN_FILE):
    sys.exit(f"❌ Train file not found at {TRAIN_FILE}\nCheck the exact file name above and update TRAIN_FILE.")
else:
    print(f"✅ Found training file: {TRAIN_FILE}")

# ✅ Step 4: Patch output directory inside your Python script
with open(TRAIN_FILE, "r", encoding="utf-8") as f:
    code = f.read()

new_code = code.replace('output_dir="./ner_results"', f'output_dir="{DRIVE_OUT}"')

with open(TRAIN_FILE, "w", encoding="utf-8") as f:
    f.write(new_code)

print("🔧 Patched output_dir →", DRIVE_OUT)

# ✅ Step 5: Confirm first 40 lines
!head -n 40 "$TRAIN_FILE"


📁 Repo files:
 - README.md
 - .git
 - data
 - notebooks
 - scripts
 - .gitignore
 - 01_test_ner.ipynb
 - requirements.txt
✅ Found training file: /content/repo/scripts/7_train_quick_ner.py
🔧 Patched output_dir → /content/drive/MyDrive/Finance-Insight/ner_results
import os
import json
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer, DataCollatorForTokenClassification
import evaluate

# --- Config ---
DATA_DIR = "data/processed/ner_auto_splits"
MODEL_NAME = "ProsusAI/finbert"
BATCH_SIZE = 2  
EPOCHS = 3
OUTPUT_DIR = "data/finbert_ner_results" 

# --- Device Check ---
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# --- Load dataset ---
dataset = load_dataset("json", data_files={
    "train": f"{DATA_DIR}/train.jsonl",
    "validation": f"{DATA_DIR}/validation.jsonl",
    "test": f"{DATA_DIR}/test.jsonl"
})

# --- Print dataset sizes ---
print("📊 Dat

In [8]:
import os, json
DATA_DIR = "/content/drive/MyDrive/Finance-Insight/data/processed/ner_auto_splits"


for fname in ("train.jsonl","validation.jsonl","test.jsonl","label_map.json"):
    path = os.path.join(DATA_DIR, fname)
    print(fname, "exists:", os.path.exists(path), " size:", os.path.getsize(path) if os.path.exists(path) else "NA")

# print small sample
with open(os.path.join(DATA_DIR, "train.jsonl"), encoding='utf-8') as f:
    for i,line in enumerate(f):
        if i>=2: break
        r=json.loads(line)
        print("sample tokens:", r.get("tokens")[:10])
        print("sample ner_tags:", r.get("ner_tags")[:10])


train.jsonl exists: True  size: 5513339
validation.jsonl exists: True  size: 520955
test.jsonl exists: True  size: 444005
label_map.json exists: True  size: 411
sample tokens: ['2011', '-', '09', '-', '15']
sample ner_tags: [0, 4, 4, 4, 4]
sample tokens: ['693']
sample ner_tags: [8]


In [ ]:
%cd /content/drive/MyDrive/Finance-Insight
!mkdir -p scripts
!mv 7_train_quick_ner.py scripts/


/content/drive/MyDrive/Finance-Insight
mv: cannot stat '7_train_quick_ner.py': No such file or directory


In [ ]:
!mkdir -p /content/drive/MyDrive/Finance-Insight/data/processed
!mv /content/drive/MyDrive/Finance-Insight/ner_auto_splits.zip /content/drive/MyDrive/Finance-Insight/data/processed/


In [ ]:
# create expected dir in Google Drive
!mkdir -p /content/drive/MyDrive/Finance-Insight/data/processed

# copy your working dataset from /content/repo → Drive
!cp -r /content/repo/data/processed/ner_auto_splits /content/drive/MyDrive/Finance-Insight/data/processed/

# verify copy worked
!ls -la /content/drive/MyDrive/Finance-Insight/data/processed/ner_auto_splits


total 6328
-rw------- 1 root root     411 Oct 31 11:41 label_map.json
-rw------- 1 root root  444005 Oct 31 11:41 test.jsonl
-rw------- 1 root root 5513339 Oct 31 11:41 train.jsonl
-rw------- 1 root root  520955 Oct 31 11:41 validation.jsonl


In [ ]:
# Search for train.jsonl in your Drive
!find /content/drive/MyDrive/Finance-Insight -type f -name "train.jsonl" -print || true


/content/drive/MyDrive/Finance-Insight/data/processed/ner_auto_splits/train.jsonl


In [ ]:
# create expected dirs and move dataset into place
mkdir -p /content/drive/MyDrive/Finance-Insight/data/processed
mv -v /content/drive/MyDrive/Finance-Insight/ner_auto_splits /content/drive/MyDrive/Finance-Insight/data/processed/ 2>/dev/null || true

# show destination
ls -la /content/drive/MyDrive/Finance-Insight/data/processed/ner_auto_splits | sed -n '1,200p'


In [ ]:
# ============================
# 🧠 FinBERT NER TRAINING RUN
# ============================

import torch, os, time

# --- 1️⃣ Confirm GPU / CPU ---
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Training will run on: {device.upper()}")

# --- 2️⃣ Change directory to your repo ---
%cd /content/drive/MyDrive/Finance-Insight

# --- 3️⃣ Check that your training script exists ---
train_path = "scripts/7_train_quick_ner.py"
if os.path.exists(train_path):
    print(f"✅ Found training script: {train_path}")
else:
    raise FileNotFoundError(f"❌ {train_path} not found! Check folder path.")

# --- 4️⃣ Create results folder (auto-saves to Drive) ---
os.makedirs("data/finbert_ner_results", exist_ok=True)
print("📂 Output dir ready:", "data/finbert_ner_results")

# --- 5️⃣ Start training ---
print("\n🧩 Starting FinBERT NER fine-tuning...\n")
start = time.time()

!python scripts/7_train_quick_ner.py

print("\n✅ Training complete! Time taken: {:.2f} minutes".format((time.time()-start)/60))

# --- 6️⃣ Show saved files ---
print("\n📁 Files saved in: data/finbert_ner_results/")
for f in os.listdir("data/finbert_ner_results"):
    print("  -", f)


Training will run on: CUDA
/content/drive/MyDrive/Finance-Insight
✅ Found training script: scripts/7_train_quick_ner.py
📂 Output dir ready: data/finbert_ner_results

🧩 Starting FinBERT NER fine-tuning...

2025-11-01 09:59:09.240552: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761991149.259962   11447 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761991149.265900   11447 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1761991149.281329   11447 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761991149.281353   11447 computation_placer.

In [ ]:
import pandas as pd
from IPython.display import display

# your metrics
metrics = {
    "Metric": ["F1-score", "Accuracy", "Precision", "Recall"],
    "Score": [0.7731, 0.9315, 0.7809, 0.7656]
}

df_metrics = pd.DataFrame(metrics)
df_metrics.style.set_caption("FinBERT NER - Test Set Metrics").set_table_styles([
    {"selector": "caption", "props": [("font-size", "16px"), ("font-weight", "bold"), ("color", "#2E86C1")]}
])
display(df_metrics)


,Metric,Score
0,F1-score,0.7731
1,Accuracy,0.9315
2,Precision,0.7809
3,Recall,0.7656


In [ ]:
df_metrics.to_csv("/content/drive/MyDrive/Finance-Insight/data/finbert_ner_results/test_metrics.csv", index=False)


In [ ]:
!ls /content/repo


01_test_ner.ipynb  data  notebooks  README.md  requirements.txt  scripts


In [ ]:
# confirm paths
import os
print("Repo exists:", os.path.exists("/content/repo"))
print("Repo top-level:", os.listdir("/content/repo")[:20] if os.path.exists("/content/repo") else "NA")
print("Results src:", os.path.exists("/content/drive/MyDrive/Finance-Insight/data/finbert_ner_results"))
print("Results dst:", os.path.exists("/content/repo/data/finbert_ner_results"))


Repo exists: True
Repo top-level: ['.gitignore', '01_test_ner.ipynb', 'scripts', 'data', '.git', 'README.md', 'notebooks', 'requirements.txt']
Results src: True
Results dst: True


In [ ]:
import os, shutil
SRC = "/content/drive/MyDrive/Finance-Insight/data/finbert_ner_results"
DST = "/content/repo/data/finbert_ner_results"

if os.path.realpath(SRC) != os.path.realpath(DST):
    if os.path.exists(DST):
        shutil.rmtree(DST)
    shutil.copytree(SRC, DST)
    print("✅ Copied results to repo/data/finbert_ner_results")
else:
    print("⚠️ Source and destination are the same — no copy needed.")


✅ Copied results to repo/data/finbert_ner_results


In [ ]:
# ✅ Commit & push results + notebook from /content/repo to GitHub
from getpass import getpass
import os, sys

REPO_DIR = "/content/repo"
GITHUB_REPO = "Suryasnata1404/Finance-Insight"  # keep as is

# ask for PAT securely
token = getpass("Paste your GitHub PAT (input hidden): ")

# basic safety checks
if not os.path.exists(REPO_DIR):
    raise SystemExit("❌ /content/repo not found. Clone first.")
print("Repo dir ok:", REPO_DIR)

# set git user (local to this clone)
os.system(f'git -C "{REPO_DIR}" config user.email "you@example.com"')
os.system(f'git -C "{REPO_DIR}" config user.name "Your Name"')

# set authenticated origin using token (temporarily)
push_url = f"https://{token}@github.com/{GITHUB_REPO}.git"
os.system(f'git -C "{REPO_DIR}" remote remove origin 2>/dev/null || true')
os.system(f'git -C "{REPO_DIR}" remote add origin {push_url}')

# ensure results folder is present
results_path = os.path.join(REPO_DIR, "data", "finbert_ner_results")
print("Results folder exists:", os.path.exists(results_path))

# add files (force in case .gitignore blocks them)
os.system(f'git -C "{REPO_DIR}" add data/finbert_ner_results -f')
os.system(f'git -C "{REPO_DIR}" add 01_test_ner.ipynb -f')

# commit
ret = os.system(f'git -C "{REPO_DIR}" commit -m "Add FinBERT NER results and notebook" || true')
print("Commit exit code (0 means commit created):", ret)

# push: try main, then master
print("Trying to push to 'main' branch...")
ret = os.system(f'git -C "{REPO_DIR}" push origin main || git -C "{REPO_DIR}" push origin master')
if ret == 0:
    print("✅ Pushed to GitHub (main/master). Check your repo now.")
else:
    print("⚠️ Push returned non-zero. If protected-branch rules exist, please push from your local machine (VSCode).")


Paste your GitHub PAT (input hidden): ··········
Repo dir ok: /content/repo
Results folder exists: False
Commit exit code (0 means commit created): 0
Trying to push to 'main' branch...
✅ Pushed to GitHub (main/master). Check your repo now.


In [ ]:
# ============================================
# ✅ COMPLETE FIX FOR FIN_TERM: Retrain with Class Weights
# ============================================
# This will fix the FIN_TERM issue by retraining with balanced class weights

import os
import json
import torch
import torch.nn as nn
from transformers import (
    AutoTokenizer,
    AutoModelForTokenClassification,
    TrainingArguments,
    Trainer,
    DataCollatorForTokenClassification
)
from datasets import load_dataset
from collections import Counter
import numpy as np
import evaluate # Import evaluate library
from seqeval.metrics import classification_report # Import classification_report here

# ============================================
# STEP 1: Load Dataset and Label Map
# ============================================
DATA_DIR = "/content/drive/MyDrive/Finance-Insight/data/processed/ner_auto_splits"

dataset = load_dataset(
    "json",
    data_files={
        "train": os.path.join(DATA_DIR, "train.jsonl"),
        "validation": os.path.join(DATA_DIR, "validation.jsonl"),
        "test": os.path.join(DATA_DIR, "test.jsonl")
    }
)

with open(os.path.join(DATA_DIR, "label_map.json"), "r") as f:
    label_info = json.load(f)
label2id = label_info["label2id"]
id2label = {int(k): v for k, v in label_info["id2label"].items()}

# Create label list for seqeval
label_list = [id2label[i] for i in sorted(id2label.keys())]

print("✅ Loaded dataset and label map")
print(f"   Train: {len(dataset['train'])} samples")
print(f"   Validation: {len(dataset['validation'])} samples")
print(f"   Test: {len(dataset['test'])} samples\n")

# ============================================
# STEP 2: Calculate Class Weights
# ============================================
print("Calculating class weights from training data...")
all_labels = []
for sample in dataset["train"]:
    # Exclude -100 (ignored tokens)
    all_labels.extend([tag for tag in sample["ner_tags"] if tag != -100])


label_counts = Counter(all_labels)
total_samples = len(all_labels)
num_classes = len(label2id)

print(f"\nLabel distribution in training data:")
for label_id in sorted(label_counts.keys()):
    label_name = id2label.get(label_id, f"ID_{label_id}")
    count = label_counts[label_id]
    percentage = (count / total_samples) * 100
    print(f"  {label_name:15s} (ID {label_id}): {count:6d} ({percentage:5.2f}%)")

# Calculate class weights (inverse frequency weighting)
class_weights = torch.zeros(num_classes)
# Ensure all labels from 0 to num_classes-1 are considered, even if not in label_counts
for label_id in range(num_classes):
    count = label_counts.get(label_id, 0) # Get count, default to 0 if label_id not in counts
    if count > 0:
        # Higher weight for rare classes (like FIN_TERM)
        class_weights[label_id] = total_samples / (num_classes * count)
    else:
        # Assign a default weight (e.g., 1.0 or average) for classes not in training data
        # Using 1.0 might be too low, let's use the mean of existing weights or a sensible default
        # For now, let's use 1.0, but be aware this might need tuning if some classes are truly missing
        class_weights[label_id] = 1.0


# Normalize weights
class_weights = class_weights / class_weights.mean() * num_classes

print(f"\n✅ Calculated class weights (higher = more important):")
for label_id in range(num_classes):
    label_name = id2label.get(label_id, f"ID_{label_id}")
    print(f"  {label_name:15s}: {class_weights[label_id]:.4f}")

# Show FIN_TERM weights specifically
# Ensure the label IDs for FIN_TERM exist in label2id before accessing
fin_term_b_id = label2id.get("B-FIN_TERM")
fin_term_i_id = label2id.get("I-FIN_TERM")
o_id = label2id.get("O")

if fin_term_b_id is not None and fin_term_i_id is not None and o_id is not None:
    fin_term_weight_b = class_weights[fin_term_b_id]
    fin_term_weight_i = class_weights[fin_term_i_id]
    o_weight = class_weights[o_id]
    print(f"\n⚠️  FIN_TERM weights: B-FIN_TERM={fin_term_weight_b:.4f}, I-FIN_TERM={fin_term_weight_i:.4f}")
    if o_weight > 0:
         print(f"   O weight: {o_weight:.4f} (B-FIN_TERM is {fin_term_weight_b/o_weight:.1f}x more important)")
    else:
        print(f"   O weight: {o_weight:.4f}")
else:
    print("\nWarning: Could not find all FIN_TERM or O labels in label map to show specific weights.")


# ============================================
# STEP 3: Tokenize Dataset
# ============================================
MODEL_NAME = "ProsusAI/finbert"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"],
        truncation=True,
        is_split_into_words=True,
        padding="max_length",
        max_length=512
    )

    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                # Only label the first token of a word
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100) # Set subsequent tokens of a word to -100
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

print("Tokenizing dataset...")
tokenized_dataset = dataset.map(tokenize_and_align_labels, batched=True)
print("✅ Dataset tokenized\n")

# ============================================
# STEP 4: Create Weighted Trainer and Metric Computation
# ============================================
class WeightedTrainer(Trainer):
    """Custom Trainer with class-weighted loss"""
    def __init__(self, class_weights, *args, **kwargs):
        super().__init__(*args, **kwargs)
        # Ensure class_weights are on the correct device
        self.class_weights = class_weights.to(self.args.device)


    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        """Compute loss with class weights, accepting additional kwargs for compatibility"""
        labels = inputs.get("labels")
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")

        # Use weighted CrossEntropyLoss
        # Ensure weights are on the same device as logits and labels
        loss_fct = nn.CrossEntropyLoss(weight=self.class_weights, ignore_index=-100)

        # Only compute loss on active labels
        active_loss = labels.view(-1) != -100
        active_logits = logits.view(-1, self.model.config.num_labels)
        active_labels = torch.where(
            active_loss,
            labels.view(-1),
            torch.tensor(loss_fct.ignore_index).type_as(labels)
        )
        loss = loss_fct(active_logits, active_labels)

        return (loss, outputs) if return_outputs else loss

# Define metric computation function
seqeval = evaluate.load("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (usually -100)
    true_predictions = [
        [id2label[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [id2label[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    # Corrected scheme name to uppercase "IOB2"
    results = seqeval.compute(predictions=true_predictions, references=true_labels, scheme="IOB2", mode="strict")
    # Return a dictionary of metrics
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }


# ============================================
# STEP 5: Load Model and Setup Training
# ============================================
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}\n")

model = AutoModelForTokenClassification.from_pretrained(
    MODEL_NAME,
    num_labels=len(label2id),
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True  # Fix: Ignore size mismatch for classifier layer
).to(device)


# Training arguments
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/Finance-Insight/data/finbert_ner_weighted",
    num_train_epochs=3,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=100,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1", # Use the 'f1' computed by compute_metrics
    greater_is_better=True,
    report_to="none",
    dataloader_num_workers=2 # Added for potentially faster data loading
)

data_collator = DataCollatorForTokenClassification(tokenizer)

# ============================================
# STEP 6: Create and Train Weighted Trainer
# ============================================
trainer = WeightedTrainer(
    class_weights=class_weights,
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"], # Corrected to access the train split
    eval_dataset=tokenized_dataset["validation"], # Corrected to access the validation split
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics # Pass the compute_metrics function
)

print("=" * 80)
print("🚀 STARTING TRAINING WITH CLASS WEIGHTS")
print("=" * 80)
print("This will retrain the model with higher weights for FIN_TERM")
print("This should significantly improve FIN_TERM detection!\n")

# Train the model
trainer.train()

print("\n✅ Training complete with class weights!")
print("📁 Model saved to:", training_args.output_dir)

# ============================================
# STEP 7: Evaluate on Test Set
# ============================================
print("\n" + "=" * 80)
print("EVALUATING ON TEST SET")
print("=" * 80)

# Run predictions on test set
predictions = trainer.predict(tokenized_dataset["test"])
pred_ids = np.argmax(predictions.predictions, axis=-1)

# Convert to label sequences for seqeval
def ids_to_label_seqs(pred_array, label_id_array):
    pred_seqs = []
    true_seqs = []
    for p_row, t_row in zip(pred_array, label_id_array):
        p_list = []
        t_list = []
        for p, t in zip(p_row, t_row):
            # Only include tokens that were not ignored (-100)
            if int(t) != -100:
                p_list.append(id2label.get(int(p), "O")) # Get label name from ID
                t_list.append(id2label.get(int(t), "O")) # Get label name from ID
        if p_list: # Only append if there are actual tokens to evaluate
            pred_seqs.append(p_list)
            true_seqs.append(t_list)
    return pred_seqs, true_seqs

pred_label_seqs, true_label_seqs = ids_to_label_seqs(pred_ids, predictions.label_ids)

print("\n=== SeqEval classification report (test set) - AFTER FIX ===")
# Ensure the label list used for the report matches the labels present in the data
# and is sorted correctly.
# seqeval.metrics.classification_report expects a list of label names.
# We can use the label_list created earlier, excluding -100 if it was implicitly included.
report_labels = [label for label in label_list if label != "-100"] # Ensure -100 is not in labels

# Check if any of the true labels are not in report_labels and add them if necessary
all_true_labels_in_test = set(tag for seq in true_label_seqs for tag in seq)
for label in all_true_labels_in_test:
    if label not in report_labels:
        report_labels.append(label)

# Sort the report_labels based on their original ID order
report_labels.sort(key=lambda x: label2id.get(x, float('inf'))) # Use get with inf for labels not in original map

print(classification_report(true_label_seqs, pred_label_seqs, digits=4, zero_division=0)) # Added zero_division=0

print("\n✅ FIN_TERM should now have improved scores!")
print("   Compare these results with the previous 0.0000 scores.")

✅ Loaded dataset and label map
   Train: 9284 samples
   Validation: 1161 samples
   Test: 1161 samples

Calculating class weights from training data...

Label distribution in training data:
  B-DATE          (ID 0):  12249 ( 3.98%)
  B-FIN_TERM      (ID 1):    289 ( 0.09%)
  B-FIN_VALUE     (ID 2):   1217 ( 0.40%)
  B-ORG           (ID 3):  11404 ( 3.70%)
  I-DATE          (ID 4):  25640 ( 8.33%)
  I-FIN_TERM      (ID 5):    319 ( 0.10%)
  I-FIN_VALUE     (ID 6):   1418 ( 0.46%)
  I-ORG           (ID 7):   9461 ( 3.07%)
  O               (ID 8): 245914 (79.87%)

✅ Calculated class weights (higher = more important):
  B-DATE         : 0.7835
  B-FIN_TERM     : 33.2081
  B-FIN_VALUE    : 7.8859
  B-ORG          : 0.8416
  I-DATE         : 0.3743
  I-FIN_TERM     : 30.0851
  I-FIN_VALUE    : 6.7681
  I-ORG          : 1.0144
  O              : 0.0390

⚠️  FIN_TERM weights: B-FIN_TERM=33.2081, I-FIN_TERM=30.0851
   O weight: 0.0390 (B-FIN_TERM is 850.9x more important)
Tokenizing dataset..

Map:   0%|          | 0/9284 [00:00<?, ? examples/s]

Map:   0%|          | 0/1161 [00:00<?, ? examples/s]

Map:   0%|          | 0/1161 [00:00<?, ? examples/s]

✅ Dataset tokenized

Using device: cuda



...

🚀 STARTING TRAINING WITH CLASS WEIGHTS
This will retrain the model with higher weights for FIN_TERM
This should significantly improve FIN_TERM detection!



Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.299700,0.399667,0.457211,0.766995,0.572908,0.846974
2,0.308200,0.410376,0.615150,0.813506,0.700558,0.905017


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.299700,0.399667,0.457211,0.766995,0.572908,0.846974
2,0.308200,0.410376,0.615150,0.813506,0.700558,0.905017
3,0.200400,0.517217,0.651280,0.807692,0.721102,0.912764



✅ Training complete with class weights!
📁 Model saved to: /content/drive/MyDrive/Finance-Insight/data/finbert_ner_weighted

EVALUATING ON TEST SET



=== SeqEval classification report (test set) - AFTER FIX ===
              precision    recall  f1-score   support

        DATE     0.8371    0.9294    0.8808      1161
    FIN_TERM     0.0663    0.3438    0.1111        32
   FIN_VALUE     0.6409    0.8198    0.7194       172
         ORG     0.4960    0.6739    0.5714       834

   micro avg     0.6385    0.8154    0.7162      2199
   macro avg     0.5101    0.6917    0.5707      2199
weighted avg     0.6812    0.8154    0.7396      2199


✅ FIN_TERM should now have improved scores!
   Compare these results with the previous 0.0000 scores.


In [17]:
# Cell 2 — Set and check paths (edit if your files are elsewhere)
import os
import json
import sys
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification
from datasets import load_dataset

# Prefer these two canonical paths — change if different
REPO_DIR = "/content/repo"   # repo if cloned into /content/repo
DRIVE_BASE = "/content/drive/MyDrive/Finance-Insight"  # your Drive copy
# Where training splits live (use whichever exists)
CANDIDATES = [
    os.path.join(REPO_DIR, "data/processed/ner_auto_splits"),
    os.path.join(DRIVE_BASE, "data/processed/ner_auto_splits"),
    os.path.join(DRIVE_BASE, "data/ner_auto_splits"),
    os.path.join(REPO_DIR, "data/ner_auto_splits"),
]

DATA_DIR = None
for c in CANDIDATES:
    if os.path.exists(c):
        DATA_DIR = c
        break
if DATA_DIR is None:
    raise FileNotFoundError("Couldn't find ner_auto_splits in expected locations. Update CANDIDATES.")


# Explicitly set the model directory to the known correct path
MODEL_DIR = os.path.join(DRIVE_BASE, "data/finbert_ner_weighted")


print("Using MODEL_DIR:", MODEL_DIR)
print("Using DATA_DIR:", DATA_DIR)
# quick file checks
for fname in ("train.jsonl","validation.jsonl","test.jsonl","label_map.json"):
    p = os.path.join(DATA_DIR, fname)
    print(fname, "exists:", os.path.exists(p), "size:", os.path.getsize(p) if os.path.exists(p) else "NA")

# Load label map
label_map_path = os.path.join(DATA_DIR, "label_map.json")
with open(label_map_path, "r", encoding="utf-8") as f:
    label_map = json.load(f)

id2label = {}
if isinstance(label_map, dict) and "id2label" in label_map:
    id2label = {int(k): v for k, v in label_map["id2label"].items()}
elif isinstance(label_map, dict) and "label2id" in label_map:
    label2id = label_map["label2id"]
    id2label = {int(v): k for k, v in label2id.items()}
else:
    found = False
    for k, v in label_map.items():
        if isinstance(v, dict) and "id" in v:
            id2label[int(v["id"])] = k
            found = True
    if not found:
        try:
            id2label = {int(k): v for k, v in label_map.items()}
        except Exception:
            raise RuntimeError("Unrecognized label_map.json format. Inspect file: " + label_map_path)

print("Loaded label map. Num labels:", len(id2label))
print("Sample id2label:", dict(list(id2label.items())[:12]))


# Load tokenizer and model
BASE_MODEL = "ProsusAI/finbert" # Assuming the base model is FinBERT
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
try:
    # Try loading from the fine-tuned model directory first
    tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR, use_fast=True)
    model = AutoModelForTokenClassification.from_pretrained(MODEL_DIR).to(DEVICE)
    print(f"✅ Model and tokenizer loaded from fine-tuned directory: {MODEL_DIR}")
except Exception as e:
    print(f"Could not load from fine-tuned directory: {e}")
    print(f"Attempting to load tokenizer from base model: {BASE_MODEL}")
    tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, use_fast=True)
    # You would typically load the fine-tuned model, so the above error indicates an issue.
    # If you intended to use the base model for some reason, load it here:
    # model = AutoModelForTokenClassification.from_pretrained(BASE_MODEL).to(DEVICE)
    raise # Re-raise the original error as loading the fine-tuned model is the goal.

Using MODEL_DIR: /content/drive/MyDrive/Finance-Insight/data/finbert_ner_weighted
Using DATA_DIR: /content/repo/data/processed/ner_auto_splits
train.jsonl exists: True size: 5504055
validation.jsonl exists: True size: 519794
test.jsonl exists: True size: 442844
label_map.json exists: True size: 388
Loaded label map. Num labels: 9
Sample id2label: {0: 'B-DATE', 1: 'B-FIN_TERM', 2: 'B-FIN_VALUE', 3: 'B-ORG', 4: 'I-DATE', 5: 'I-FIN_TERM', 6: 'I-FIN_VALUE', 7: 'I-ORG', 8: 'O'}
Could not load from fine-tuned directory: stat: path should be string, bytes, os.PathLike or integer, not NoneType
Attempting to load tokenizer from base model: ProsusAI/finbert


TypeError: stat: path should be string, bytes, os.PathLike or integer, not NoneType

In [3]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
import os

# Define paths
MODEL_DIR = "/content/drive/MyDrive/Finance-Insight/data/finbert_ner_weighted"
CHECKPOINT_DIR = os.path.join(MODEL_DIR, "checkpoint-9284")

# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(CHECKPOINT_DIR, use_fast=True)
model = AutoModelForTokenClassification.from_pretrained(CHECKPOINT_DIR)

print(f"✅ Loaded model from: {CHECKPOINT_DIR}")


✅ Loaded model from: /content/drive/MyDrive/Finance-Insight/data/finbert_ner_weighted/checkpoint-9284


In [4]:
from datasets import load_dataset

DATA_DIR = "/content/drive/MyDrive/Finance-Insight/data/processed/ner_auto_splits"

dataset = load_dataset(
    "json",
    data_files={
        "train": os.path.join(DATA_DIR, "train.jsonl"),
        "validation": os.path.join(DATA_DIR, "validation.jsonl"),
        "test": os.path.join(DATA_DIR, "test.jsonl"),
    }
)

print(dataset)


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags', 'text'],
        num_rows: 9284
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags', 'text'],
        num_rows: 1161
    })
    test: Dataset({
        features: ['tokens', 'ner_tags', 'text'],
        num_rows: 1161
    })
})


In [6]:
!pip install -q seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [7]:
# Cell — Fix label_map parsing, print ner-tag counts per split, run prediction & seqeval report
import os
import json
import numpy as np
from collections import Counter
from transformers import TrainingArguments, Trainer, DataCollatorForTokenClassification
from seqeval.metrics import classification_report
from datasets import Dataset
import torch
from transformers import AutoTokenizer # Import AutoTokenizer

# Assume tokenizer and model are already loaded from previous cells
# tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR, use_fast=True)
# model = AutoModelForTokenClassification.from_pretrained(MODEL_DIR)

# ---------- 1) Load/parse label_map.json robustly ----------
label_map_path = os.path.join(DATA_DIR, "label_map.json")
with open(label_map_path, "r", encoding="utf-8") as f:
    label_map = json.load(f)

# common shapes:
# a) {"label2id": {"O":0, "B-ORG":1,...}, "id2label": {"0":"O",...}}
# b) {"O": {"id":0}, ...}  (less common)
# c) {"id2label": {"0":"O", ...}} or {"label2id": {...}}
id2label = {}
if isinstance(label_map, dict) and "id2label" in label_map:
    # id2label stored directly (keys probably strings)
    id2label = {int(k): v for k, v in label_map["id2label"].items()}
elif isinstance(label_map, dict) and "label2id" in label_map:
    # invert label2id
    label2id = label_map["label2id"]
    id2label = {int(v): k for k, v in label2id.items()}
else:
    # fallback: try to find a mapping inside the file
    # if file is flat mapping label->id or id->label
    # try label->dict with "id"
    found = False
    for k, v in label_map.items():
        if isinstance(v, dict) and "id" in v:
            id2label[int(v["id"])] = k
            found = True
    if not found:
        # try invert if keys are numeric strings
        try:
            id2label = {int(k): v for k, v in label_map.items()}
        except Exception:
            raise RuntimeError("Unrecognized label_map.json format. Inspect file: " + label_map_path)


# final label list sorted by id
label_list = [id2label[i] for i in sorted(id2label.keys())]
print("Loaded label map. Num labels:", len(label_list))
print("Sample id2label:", dict(list(id2label.items())[:12]))


Loaded label map. Num labels: 9
Sample id2label: {0: 'B-DATE', 1: 'B-FIN_TERM', 2: 'B-FIN_VALUE', 3: 'B-ORG', 4: 'I-DATE', 5: 'I-FIN_TERM', 6: 'I-FIN_VALUE', 7: 'I-ORG', 8: 'O'}


In [23]:

# ---------- 2) Print token counts (ner tag counts) per split ----------
def count_tag_ids(split):
    cnt = Counter()
    for arr in split["ner_tags"]:
        cnt.update(arr)
    return cnt

for split_name in ("train","validation","test"):
    cnt = count_tag_ids(dataset[split_name])
    # convert to readable mapping
    readable = { (id2label[i] if i in id2label else str(i)): c for i,c in cnt.items() }
    total_tokens = sum(cnt.values())
    print(f"\n--- {split_name} --- rows: {len(dataset[split_name])} tokens_total: {total_tokens}")
    # show top 12 counts (exclude -100 if present)
    if -100 in cnt:
        print("  ignored tokens (-100):", cnt[-100])
    top = Counter({k:v for k,v in cnt.items() if k != -100}).most_common(12)
    for tid, c in top:
        lbl = id2label.get(tid, str(tid))
        print(f"   {lbl:15s} ({tid}): {c}")

# ---------- Preprocess data for model prediction ----------
# Function to tokenize and align labels
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

# Apply preprocessing to the test dataset
tokenized_dataset = dataset["test"].map(tokenize_and_align_labels, batched=True)


# ---------- 3) Prepare Trainer for prediction ----------
# minimal TrainingArguments for predict
args = TrainingArguments(output_dir="./tmp_eval", per_device_eval_batch_size=16, report_to="none")
data_collator = DataCollatorForTokenClassification(tokenizer)

trainer = Trainer(model=model, args=args, tokenizer=tokenizer, data_collator=data_collator) # Pass the model and tokenizer loaded earlier

# ---------- 4) Run prediction on test split ----------
print("\nRunning prediction on test split (this may take a bit)...")
pred_out = trainer.predict(tokenized_dataset)   # Use the tokenized dataset
logits = pred_out.predictions  # shape (num_examples, seq_len, num_labels)
label_ids = pred_out.label_ids

pred_ids = np.argmax(logits, axis=-1)

# ---------- 5) Convert ids -> label strings (ignore -100) ----------
def ids_to_label_seqs(pred_array, label_id_array):
    pred_seqs = []
    true_seqs = []
    for p_row, t_row in zip(pred_array, label_id_array):
        p_list = []
        t_list = []
        for p, t in zip(p_row, t_row):
            if int(t) == -100:
                continue
            p_list.append(id2label.get(int(p), "O"))
            t_list.append(id2label.get(int(t), "O"))
        pred_seqs.append(p_list)
        true_seqs.append(t_list)
    return pred_seqs, true_seqs

pred_label_seqs, true_label_seqs = ids_to_label_seqs(pred_ids, label_ids)
print("Converted predictions -> BIO labels. Examples:", len(pred_label_seqs))

# ---------- 6) SeqEval classification report ----------
print("\n=== SeqEval classification report (test set) ===")
print(classification_report(true_label_seqs, pred_label_seqs, digits=4))


--- train --- rows: 9284 tokens_total: 307911
   O               (8): 245914
   I-DATE          (4): 25640
   B-DATE          (0): 12249
   B-ORG           (3): 11404
   I-ORG           (7): 9461
   I-FIN_VALUE     (6): 1418
   B-FIN_VALUE     (2): 1217
   I-FIN_TERM      (5): 319
   B-FIN_TERM      (1): 289

--- validation --- rows: 1161 tokens_total: 27825
   O               (8): 21911
   I-DATE          (4): 2336
   B-ORG           (3): 1140
   B-DATE          (0): 1139
   I-ORG           (7): 910
   I-FIN_VALUE     (6): 159
   B-FIN_VALUE     (2): 148
   B-FIN_TERM      (1): 43
   I-FIN_TERM      (5): 39

--- test --- rows: 1161 tokens_total: 23744
   O               (8): 17881
   I-DATE          (4): 2607
   B-DATE          (0): 1205
   B-ORG           (3): 880
   I-ORG           (7): 750
   I-FIN_VALUE     (6): 188
   B-FIN_VALUE     (2): 172
   B-FIN_TERM      (1): 34
   I-FIN_TERM      (5): 27


Map:   0%|          | 0/1161 [00:00<?, ? examples/s]

/tmp/ipython-input-4228043959.py:54: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model=model, args=args, tokenizer=tokenizer, data_collator=data_collator) # Pass the model and tokenizer loaded earlier



Running prediction on test split (this may take a bit)...


Converted predictions -> BIO labels. Examples: 1161

=== SeqEval classification report (test set) ===
              precision    recall  f1-score   support

        DATE     0.8512    0.9216    0.8850      1161
    FIN_TERM     0.0522    0.3750    0.0916        32
   FIN_VALUE     0.6198    0.8721    0.7246       172
         ORG     0.4422    0.6739    0.5340       834

   micro avg     0.5980    0.8158    0.6901      2199
   macro avg     0.4914    0.7106    0.5588      2199
weighted avg     0.6664    0.8158    0.7278      2199



In [ ]:
import torch

def predict_ner(text, tokenizer, model, id2label):
    # Tokenize the input text
    tokens = text.split() # Simple split, can be improved for better tokenization

    # Prepare input for the model
    tokenized_inputs = tokenizer(
        tokens,
        is_split_into_words=True,
        return_tensors="pt",
        truncation=True,
        padding=True
    )

    # Move input to the same device as the model
    device = model.device
    input_ids = tokenized_inputs["input_ids"].to(device)
    attention_mask = tokenized_inputs["attention_mask"].to(device)

    # Get model predictions
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1).squeeze().tolist()

    # Align predictions with original tokens
    word_ids = tokenized_inputs.word_ids(batch_index=0)
    aligned_tags = []
    previous_word_idx = None
    for word_idx, pred_id in zip(word_ids, predictions):
        if word_idx is not None and word_idx != previous_word_idx:
            aligned_tags.append(id2label.get(pred_id, "O"))
        previous_word_idx = word_idx

    return list(zip(tokens, aligned_tags))

# Example Usage:
text_input = "Apple's revenue in Q3 was $83.4 billion."
predicted_entities = predict_ner(text_input, tokenizer, model, id2label)

print("Input Text:", text_input)
print("Predicted Entities:")
for token, tag in predicted_entities:
    print(f"{token}: {tag}")

Input Text: Apple's revenue in Q3 was $83.4 billion.
Predicted Entities:
Apple's: B-ORG
revenue: O
in: O
Q3: B-DATE
was: O
$83.4: O
billion.: I-FIN_VALUE


In [25]:
# --- Step 1: Load Everything ---
import json
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification
from collections import defaultdict, Counter
import os

# Paths - Adjust these if your paths are different
# Use the path where the model checkpoint was saved during training
MODEL_PATH = "/content/drive/MyDrive/Finance-Insight/data/finbert_ner_weighted/checkpoint-9284" #"ProsusAI/finbert"#
# Use the path to your test data and label map
TEST_FILE = "/content/drive/MyDrive/Finance-Insight/data/processed/ner_auto_splits/test.jsonl"
LABEL_MAP_FILE = "/content/drive/MyDrive/Finance-Insight/data/processed/ner_auto_splits/label_map.json"

# Load model and tokenizer
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

try:
    model = AutoModelForTokenClassification.from_pretrained(MODEL_PATH).to(device)
    tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
    print("✅ Model and tokenizer loaded successfully.")
except Exception as e:
    print(f"Error loading model or tokenizer: {e}")
    print(f"Please verify the MODEL_PATH: {MODEL_PATH}")
    # Optionally, you could fall back to loading from the base model if needed
    # model = AutoModelForTokenClassification.from_pretrained("ProsusAI/finbert").to(device)
    # tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
    raise # Re-raise the error as we need the fine-tuned model

model.eval()

# Load label map
try:
    with open(LABEL_MAP_FILE, 'r', encoding='utf-8') as f:
        label_map = json.load(f)

    id2label = {}
    if isinstance(label_map, dict) and "id2label" in label_map:
        id2label = {int(k): v for k, v in label_map["id2label"].items()}
    elif isinstance(label_map, dict) and "label2id" in label_map:
        label2id = label_map["label2id"]
        id2label = {int(v): k for k, v in label2id.items()}
    else:
         try:
             id2label = {int(k): v for k, v in label_map.items()}
         except Exception:
             raise RuntimeError("Unrecognized label_map.json format. Inspect file: " + LABEL_MAP_FILE)

    print(f"✅ Label map loaded. Num labels: {len(id2label)}")
    print("Sample id2label:", dict(list(id2label.items())[:12]))

except Exception as e:
    print(f"Error loading label map: {e}")
    print(f"Please verify the LABEL_MAP_FILE: {LABEL_MAP_FILE}")
    raise


# Load test data
test_samples = []
try:
    with open(TEST_FILE, 'r', encoding='utf-8') as f:
        for line in f:
            test_samples.append(json.loads(line))
    print(f"✅ Loaded {len(test_samples)} test samples.")

except Exception as e:
    print(f"Error loading test data: {e}")
    print(f"Please verify the TEST_FILE: {TEST_FILE}")
    raise

Using device: cuda
✅ Model and tokenizer loaded successfully.
✅ Label map loaded. Num labels: 9
Sample id2label: {0: 'B-DATE', 1: 'B-FIN_TERM', 2: 'B-FIN_VALUE', 3: 'B-ORG', 4: 'I-DATE', 5: 'I-FIN_TERM', 6: 'I-FIN_VALUE', 7: 'I-ORG', 8: 'O'}
✅ Loaded 1161 test samples.


In [26]:
# --- Step 2: Run Inference on Test Set ---

def predict_tags(tokens, tokenizer, model, id2label, device):
    """Run inference on a list of tokens and return aligned predictions."""
    tokenized = tokenizer(
        tokens,
        is_split_into_words=True,
        return_tensors="pt",
        truncation=True,
        padding=True
    )

    input_ids = tokenized["input_ids"].to(device)
    attention_mask = tokenized["attention_mask"].to(device)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)

    predictions = torch.argmax(outputs.logits, dim=-1).squeeze().tolist()

    # Align predictions to original tokens
    word_ids = tokenized.word_ids(batch_index=0)
    aligned_predictions = []
    previous_word_idx = None

    # Handle the case where predictions is a single integer (for very short inputs)
    if isinstance(predictions, int):
        predictions = [predictions]

    for word_idx, pred_id in zip(word_ids, predictions):
        if word_idx is not None and word_idx != previous_word_idx:
            # Ensure pred_id is a valid key in id2label
            aligned_predictions.append(id2label.get(pred_id, "O")) # Default to "O" if id is not found
        previous_word_idx = word_idx

    # Since padding can add extra tokens, we need to ensure the number of predictions
    # matches the number of original tokens. We can truncate or align based on word_ids.
    # The current logic aligns based on word_ids, so we should return one tag per original token.
    # We need to make sure we don't have more aligned_predictions than original tokens.
    return aligned_predictions[:len(tokens)]


# Run inference on all test samples
test_predictions = []
for i, sample in enumerate(test_samples):
    if i % 100 == 0:
        print(f"Processing sample {i}/{len(test_samples)}")
    tokens = sample["tokens"]
    predicted_tags = predict_tags(tokens, tokenizer, model, id2label, device)
    test_predictions.append(predicted_tags)

print("\nFinished running inference on the test set.")
print(f"Generated predictions for {len(test_predictions)} samples.")

Processing sample 0/1161
Processing sample 100/1161
Processing sample 200/1161
Processing sample 300/1161
Processing sample 400/1161
Processing sample 500/1161
Processing sample 600/1161
Processing sample 700/1161
Processing sample 800/1161
Processing sample 900/1161
Processing sample 1000/1161
Processing sample 1100/1161

Finished running inference on the test set.
Generated predictions for 1161 samples.


In [ ]:
# --- Step 3: Perform Token-Level Analysis ---

# Flatten the lists of true and predicted tags
true_tags_flat = [tag for sample in test_samples for tag in sample["ner_tags"]]
predicted_tags_flat = [tag_list for pred_sample in test_predictions for tag_list in pred_sample]

# Convert numerical tags to labels using id2label
true_labels_flat = [id2label.get(tag_id, "O") for tag_id in true_tags_flat]
# predicted_labels_flat = [id2label.get(tag_id, "O") for tag_id in predicted_tags_flat]

# We need to align the predicted tags with the true tags,
# considering that the inference function returns one tag per original token.
# The `test_predictions` list already contains lists of predicted tags aligned to original tokens.
# We need to flatten this structure correctly.

# Flatten the predicted tags, ensuring alignment with true tags
# (This assumes the predict_tags function returns a list of the same length as the input tokens)
predicted_labels_flat = []
for i, sample in enumerate(test_samples):
    # Ensure predicted_tags has the same length as original tokens
    predicted_labels_flat.extend(test_predictions[i][:len(sample["tokens"])])

# Ensure both lists have the same length for comparison
min_len = min(len(true_labels_flat), len(predicted_labels_flat))
true_labels_flat = true_labels_flat[:min_len]
predicted_labels_flat = predicted_labels_flat[:min_len]


# Calculate metrics per token type
token_metrics = defaultdict(lambda: {"correct": 0, "predicted": 0, "actual": 0})

for true_tag, predicted_tag in zip(true_labels_flat, predicted_labels_flat):
    token_metrics[true_tag]["actual"] += 1
    token_metrics[predicted_tag]["predicted"] += 1
    if true_tag == predicted_tag:
        token_metrics[true_tag]["correct"] += 1

# Calculate precision, recall, and f1-score for each token type
results = {}
for tag, counts in token_metrics.items():
    correct = counts["correct"]
    predicted = counts["predicted"]
    actual = counts["actual"]

    precision = correct / predicted if predicted > 0 else 0
    recall = correct / actual if actual > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    results[tag] = {
        "failed_tokens": actual - correct, # Number of tokens where the true tag was this tag, but prediction was different
        "actual_count": actual, # Total count of this true tag
        "predicted_count": predicted, # Total count of this predicted tag
        "correct_count": correct, # Count of correct predictions for this tag
        "precision": precision,
        "recall": recall,
        "f1-score": f1
    }

# Print the results
print("--- Token-Level Error Analysis ---")
# Sort results by F1-score or failed tokens for easier analysis
sorted_results = sorted(results.items(), key=lambda item: item[1]["f1-score"], reverse=True)

for tag, metrics in sorted_results:
    print(f"\nToken Type: {tag}")
    print(f"  Failed Tokens: {metrics['failed_tokens']}")
    print(f"  Actual Count: {metrics['actual_count']}")
    print(f"  Predicted Count: {metrics['predicted_count']}")
    print(f"  Correct Count: {metrics['correct_count']}")
    print(f"  Precision: {metrics['precision']:.4f}")
    print(f"  Recall: {metrics['recall']:.4f}")
    print(f"  F1-score: {metrics['f1-score']:.4f}")

--- Token-Level Error Analysis ---

Token Type: O
  Failed Tokens: 4587
  Actual Count: 16619
  Predicted Count: 15245
  Correct Count: 12032
  Precision: 0.7892
  Recall: 0.7240
  F1-score: 0.7552

Token Type: I-DATE
  Failed Tokens: 1932
  Actual Count: 2462
  Predicted Count: 2563
  Correct Count: 530
  Precision: 0.2068
  Recall: 0.2153
  F1-score: 0.2109

Token Type: B-FIN_VALUE
  Failed Tokens: 126
  Actual Count: 158
  Predicted Count: 230
  Correct Count: 32
  Precision: 0.1391
  Recall: 0.2025
  F1-score: 0.1649

Token Type: B-DATE
  Failed Tokens: 945
  Actual Count: 1130
  Predicted Count: 1222
  Correct Count: 185
  Precision: 0.1514
  Recall: 0.1637
  F1-score: 0.1573

Token Type: I-ORG
  Failed Tokens: 520
  Actual Count: 652
  Predicted Count: 1048
  Correct Count: 132
  Precision: 0.1260
  Recall: 0.2025
  F1-score: 0.1553

Token Type: I-FIN_VALUE
  Failed Tokens: 142
  Actual Count: 174
  Predicted Count: 241
  Correct Count: 32
  Precision: 0.1328
  Recall: 0.1839
  F

In [ ]:
# OFFSET-AWARE NER SPAN EXTRACTOR (no overlap with your existing predict_ner)
import torch

def ner_spans_with_offsets(text: str):
    model.eval()
    with torch.inference_mode():
        enc = tokenizer(
            text,
            return_tensors="pt",
            return_offsets_mapping=True,
            truncation=True
        )
        offsets = enc.pop("offset_mapping")[0].tolist()
        enc = {k: v.to(model.device) for k, v in enc.items()}
        logits = model(**enc).logits[0]                # [seq_len, num_labels]
        probs = logits.softmax(-1)
        pred_ids = probs.argmax(-1).tolist()

    # prefer model.config.id2label if available
    _id2label = getattr(getattr(model, "config", object()), "id2label", None) or id2label

    spans, current = [], None
    for i, pid in enumerate(pred_ids):
        s, e = offsets[i]
        if (s, e) == (0, 0):
            continue  # special tokens
        lab = _id2label.get(pid, "O")
        if lab.startswith("B-"):
            if current: spans.append(current)
            current = {"label": lab[2:], "start": s, "end": e, "score": probs[i, pid].item()}
        elif lab.startswith("I-") and current and current["label"] == lab[2:]:
            current["end"] = e
            current["score"] = max(current["score"], probs[i, pid].item())
        else:
            if current: spans.append(current)
            current = None
    if current: spans.append(current)

    for sp in spans:
        sp["text"] = text[sp["start"]:sp["end"]]
    return spans


In [8]:
# New helper imports for Milestone 3
import re
from typing import List, Dict, Any


In [9]:
def normalize_money(text: str):
    """Normalize money strings like '$12B' → {'value': 12.0, 'unit': 'B', 'currency': 'USD'}."""
    pattern = r"\$?\s?([\d\.]+)\s?(billion|million|trillion|bn|m|t)?"
    m = re.search(pattern, text, flags=re.IGNORECASE)
    if not m:
        return {"raw": text}
    value = float(m.group(1))
    unit = (m.group(2) or "").lower()
    mult = {"billion":1e9,"bn":1e9,"million":1e6,"m":1e6,"trillion":1e12,"t":1e12}.get(unit,1)
    return {"raw": text, "value": value*mult, "unit": unit, "currency": "USD"}

def normalize_percent(text: str):
    """Normalize percent strings like '5%' → 0.05"""
    m = re.search(r"([\d\.]+)%", text)
    return {"raw": text, "value": float(m.group(1))/100 if m else None}


In [35]:
def user_defined_extraction(text: str, user_entities: List[str]) -> Dict[str, List[Dict[str, Any]]]:
    """Extract user-specified financial entities using regex rules."""
    results = {ent: [] for ent in user_entities}

    if "market_cap" in user_entities:
        for match in re.findall(r"market cap(?:italization)?(?: is| of)? \$?\d+(?:\.\d+)? ?(?:billion|million|trillion)?", text, flags=re.I):
            results["market_cap"].append(normalize_money(match))

    if "EPS" in user_entities:
        for match in re.findall(r"EPS(?: is| of)? \$?\d+(?:\.\d+)?", text, flags=re.I):
            results["EPS"].append(normalize_money(match))

    if "revenue_growth" in user_entities:
    # Handles: "Revenue growth rose by 8%", "Sales grew 8%", "Revenue increased 8%"
        for pct in re.findall(r"(?:revenue|sales)\s*(?:growth|grew|rose|increased|decreased|fell)\s*(?:by\s*)?(\d{1,3}(?:\.\d+)?)\s*%", text, flags=re.I):
            results["revenue_growth"].append(normalize_percent(pct + "%"))

    if "stock_price_trend" in user_entities:
        # Handles: "stock price rose 5%", "shares fell by 3%", "stock decreased 2%"
        for m in re.findall(r"(?:stock price|shares|stock)\s+(?:rose|fell|increased|decreased|gained|lost|up|down)\s+(?:by\s*)?\d{1,3}(?:\.\d+)?\s*%", text, flags=re.I):
            results["stock_price_trend"].append({"raw": m})

    return results


In [36]:
def detect_financial_events(text: str, event_types: List[str]) -> Dict[str, List[str]]:
    """Detect high-level financial events by keywords."""
    key_map = {
        "M&A": ["merger","acquisition","acquire","merged","acquired","buyout"],
        "IPO": ["ipo","initial public offering","went public"],
        "earnings_call": ["earnings call","quarterly results","conference call"],
        "dividend": ["dividend declared","dividend announced","dividend payout"],
    }
    events = {et: [] for et in event_types}
    for sent in re.split(r'(?<=[.!?]) +', text):
        low = sent.lower()
        for et in event_types:
            if any(k in low for k in key_map.get(et, [])):
                events[et].append(sent.strip())
    return events


In [37]:
sample_text = """
The company's market capitalization is $12 billion. EPS is $3.50 this quarter.
Revenue growth rose by 8%. The stock price rose 5% after the IPO announcement.
"""

user_entities = ["market_cap", "EPS", "revenue_growth", "stock_price_trend"]
event_types = ["M&A", "IPO", "earnings_call", "dividend"]

# NER already available from your model — we just use rule layer here
rule_results = user_defined_extraction(sample_text, user_entities)
events = detect_financial_events(sample_text, event_types)

print("Rule-based entities:\n", rule_results)
print("\nDetected events:\n", events)


Rule-based entities:
 {'market_cap': [{'raw': 'market capitalization is $12 billion', 'value': 12000000000.0, 'unit': 'billion', 'currency': 'USD'}], 'EPS': [{'raw': 'EPS is $3.50', 'value': 3.5, 'unit': '', 'currency': 'USD'}], 'revenue_growth': [], 'stock_price_trend': [{'raw': 'stock price rose 5%'}]}

Detected events:
 {'M&A': [], 'IPO': ['The stock price rose 5% after the IPO announcement.'], 'earnings_call': [], 'dividend': []}


In [38]:
# Runs your fine-tuned NER to get character-accurate spans + confidence.
import torch

def ner_infer(text: str, conf_threshold: float = 0.50):
    model.eval()
    with torch.inference_mode():
        enc = tokenizer(
            text,
            return_tensors="pt",
            return_offsets_mapping=True,
            truncation=True
        )
        enc = {k: v.to(model.device) for k, v in enc.items()}
        offsets = enc["offset_mapping"][0].tolist()
        logits = model(**{k: v for k, v in enc.items() if k != "offset_mapping"}).logits[0]
        probs = logits.softmax(-1)

    spans = []
    current = None
    for i, (s, e) in enumerate(offsets):
        if s == e:  # special tokens
            continue
        pred_id = int(probs[i].argmax().item())
        label = id2label.get(pred_id, "O")
        score = float(probs[i, pred_id].item())

        if label.startswith("B-") and score >= conf_threshold:
            if current:
                spans.append(current)
            current = {"label": label[2:], "start": s, "end": e, "score": score}
        elif label.startswith("I-") and current and current["label"] == label[2:]:
            current["end"] = e
            current["score"] = max(current["score"], score)
        else:
            if current:
                spans.append(current)
            current = None
    if current:
        spans.append(current)

    for sp in spans:
        sp["text"] = text[sp["start"]:sp["end"]]
    return spans


In [39]:
# Maps model labels → user-visible entities; edit/extend to your label set.
LABEL_TO_ENTITY = {
    "MARKET_CAP": "market_cap",
    "EPS_VALUE": "EPS",
    "REVENUE_GROWTH": "revenue_growth",
    "PRICE_TREND": "stock_price_trend",
    # add more: "NET_INCOME": "net_income", etc.
}

def map_ner_to_user_entities(ner_spans, user_entities):
    out = {e: [] for e in user_entities}
    for sp in ner_spans:
        ent = LABEL_TO_ENTITY.get(sp["label"])
        if ent in user_entities:
            out[ent].append({
                "source": "ner",
                "text": sp["text"],
                "start": sp["start"],
                "end": sp["end"],
                "confidence": sp["score"]
            })
    return out


In [40]:
# Combines NER results with your existing rule-based results; removes duplicates.
def merge_extractions(text: str, user_entities, conf_threshold: float = 0.50):
    ner_spans = ner_infer(text, conf_threshold=conf_threshold)
    ner_dict = map_ner_to_user_entities(ner_spans, user_entities)
    rule_dict = user_defined_extraction(text, user_entities)  # <-- already defined earlier

    merged = {e: [] for e in user_entities}

    # helper: make a simple dedup key on lowercased text (and value if present)
    def k(v):
        if isinstance(v, dict) and "text" in v:
            return ("ner", v["text"].strip().lower())
        if isinstance(v, dict) and "raw" in v:
            return ("rule", v["raw"].strip().lower())
        return ("rule", str(v).strip().lower())

    seen = set()

    # add NER first (prefer model spans), then rules
    for e in user_entities:
        for v in ner_dict.get(e, []):
            key = (e, k(v))
            if key not in seen:
                merged[e].append(v)
                seen.add(key)
        for v in rule_dict.get(e, []):
            key = (e, k(v))
            if key not in seen:
                # ensure a consistent shape for rule entries
                if isinstance(v, dict):
                    v = {"source": "rule", **v}
                else:
                    v = {"source": "rule", "raw": str(v)}
                merged[e].append(v)
                seen.add(key)

    return merged


In [41]:
# Example usage: change text or pipe real 10-K/10-Q/news content here.
text = """
The company's market capitalization is $12 billion. EPS is $3.50 this quarter.
Revenue growth rose by 8%. The stock price rose 5% after the IPO announcement.
"""

user_entities = ["market_cap", "EPS", "revenue_growth", "stock_price_trend"]
hybrid = merge_extractions(text, user_entities, conf_threshold=0.50)
print(hybrid)


{'market_cap': [{'source': 'rule', 'raw': 'market capitalization is $12 billion', 'value': 12000000000.0, 'unit': 'billion', 'currency': 'USD'}], 'EPS': [{'source': 'rule', 'raw': 'EPS is $3.50', 'value': 3.5, 'unit': '', 'currency': 'USD'}], 'revenue_growth': [], 'stock_price_trend': [{'source': 'rule', 'raw': 'stock price rose 5%'}]}


In [42]:
# Fixed: robust month lookup supports both full ("June") and abbreviated ("Jun") names.
import re
import calendar
from datetime import datetime
from typing import List, Dict, Any, Optional, Tuple

# Build month map from calendar (handles "june" and "jun")
_MONTHS = {}
for i in range(1, 13):
    _MONTHS[calendar.month_name[i].lower()] = i
    _MONTHS[calendar.month_abbr[i].lower()] = i

def _parse_date_from_sentence(sent: str) -> Optional[datetime]:
    # very small date parser: "June 5, 2024", "Jun 2024", "Q1 2024", "2024"
    s = sent.strip()
    # Month D, YYYY  | Month YYYY  (day optional)
    m = re.search(
        r"(Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|"
        r"Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)\s+(\d{1,2})?,?\s+(\d{4})",
        s, re.I
    )
    if m:
        key = m.group(1).lower()
        month = _MONTHS.get(key) or _MONTHS.get(key[:3])
        day = int(m.group(2)) if m.group(2) else 1
        year = int(m.group(3))
        return datetime(year, month, day)

    # Qn YYYY
    m = re.search(r"\bQ([1-4])\s+(\d{4})\b", s, re.I)
    if m:
        q, year = int(m.group(1)), int(m.group(2))
        month = (q - 1) * 3 + 1
        return datetime(year, month, 1)

    # YYYY only
    m = re.search(r"\b(20\d{2}|19\d{2})\b", s)
    if m:
        return datetime(int(m.group(1)), 1, 1)

    return None

_EVENT_KEYWORDS = {
    "M&A": ["merger","acquisition","acquire","merged","acquired","buyout"],
    "IPO": ["ipo","initial public offering","went public","listing","listed on"],
    "stock_split": ["stock split","share split","split shares"],
    "earnings_call": ["earnings call","quarterly results","earnings report","conference call"],
    "dividend": ["dividend declared","dividend announced","dividend payout","dividend yield"],
    "guidance": ["guidance","outlook","revenue forecast","earnings guidance"],
    "rating_change": ["credit rating","downgrade","upgrade","rating agency"],
}

def detect_financial_events(
    text: str,
    event_types: List[str],
    timeframe: Optional[Tuple[Optional[datetime], Optional[datetime]]] = None
) -> Dict[str, List[Dict[str, Any]]]:
    """Keyword-based event detection; optionally filters events by sentence date if found."""
    start, end = timeframe or (None, None)
    results = {et: [] for et in event_types}
    sentences = re.split(r'(?<=[.!?])\s+', text.strip())

    for sent in sentences:
        low = sent.lower()
        for et in event_types:
            if any(k in low for k in _EVENT_KEYWORDS.get(et, [])):
                d = _parse_date_from_sentence(sent)
                if start and d and d < start:
                    continue
                if end and d and d > end:
                    continue
                results[et].append({"sentence": sent.strip(), "date": d.isoformat() if d else None})
    return results


In [34]:
def extract_tickers(text: str) -> List[str]:
    """
    Extract likely tickers with simple context rules:
    - Allow patterns near the words 'ticker' or 'symbol'
    - Allow uppercase tickers inside parentheses, e.g., (AAPL)
    - Drop common finance terms like IPO, EPS, PE, ROE, etc.
    """
    STOP = {"IPO","EPS","PE","NAV","ROE","ROI","CAGR","EBITDA","EPS","PV","IRR","DCF","FCF"}
    seen, out = set(), []

    # Contextual: near 'ticker'/'symbol' OR inside parentheses
    for m in re.finditer(r'(\()?(?P<t>[A-Z]{1,5}(?:\.[A-Z])?)(\))?', text):
        t = m.group("t")
        if t in STOP:
            continue
        ctx = text[max(0, m.start()-20): m.end()+20].lower()
        in_paren = bool(m.group(1)) or text[m.end()-1:m.end()] == ")"
        near_hint = ("ticker" in ctx) or ("symbol" in ctx)
        if in_paren or near_hint:
            if t not in seen:
                seen.add(t); out.append(t)

    # Explicit "Ticker: XXX" or "Symbol - XXX"
    for m in re.finditer(r'(?:ticker|symbol)\s*[:\-]\s*([A-Z]{1,5}(?:\.[A-Z])?)', text, re.I):
        t = m.group(1).upper()
        if t not in STOP and t not in seen:
            seen.add(t); out.append(t)

    return out


In [43]:
# pip install yfinance
import yfinance as yf


In [44]:

def verify_with_financial_db(extracted: Dict[str, List[dict]], text: str) -> Dict[str, Any]:
    """
    Verifies/enriches using Yahoo Finance:
    - Finds tickers in text.
    - For each ticker, fetches last price and longName (if available).
    """
    verified = {"tickers": []}
    for t in extract_tickers(text):
        try:
            tk = yf.Ticker(t)
            info = tk.info or {}
            price = info.get("regularMarketPrice")
            name = info.get("longName") or info.get("shortName")
            if price is not None:
                verified["tickers"].append({"ticker": t, "price": price, "name": name or "N/A"})
        except Exception:
            # ignore invalid tickers / network errors
            pass
    # You can also cross-check values in `extracted` against fetched fundamentals if needed.
    return verified


In [45]:
def analyze_text(text: str,
                 user_entities: List[str],
                 event_types: List[str],
                 conf_threshold: float = 0.50,
                 timeframe: Optional[Tuple[Optional[datetime], Optional[datetime]]] = None):
    """
    Runs: Hybrid entity extraction → Event detection (optional timeframe) → Yahoo verify.
    """
    entities = merge_extractions(text, user_entities, conf_threshold=conf_threshold)  # uses your existing hybrid
    events = detect_financial_events(text, event_types, timeframe=timeframe)
    verified = verify_with_financial_db(entities, text)
    return {"entities": entities, "events": events, "verified": verified}


In [46]:
demo_text = """
ApexTech Solutions Ltd. announced an IPO on June 5, 2024.
The company's market capitalization is $12 billion. EPS is $3.50 this quarter.
Revenue growth rose by 8%. The stock price rose 5%. Ticker mentioned: AAPL.
"""

user_entities = ["market_cap", "EPS", "revenue_growth", "stock_price_trend"]
event_types = ["IPO", "M&A", "earnings_call", "dividend"]

# Example timeframe: only include events from 2024-01-01 to 2024-12-31
start_dt = datetime(2010,1,1); end_dt = datetime(2024,12,31)
result = analyze_text(demo_text, user_entities, event_types, conf_threshold=0.5, timeframe=(start_dt, end_dt))
print(result)


{'entities': {'market_cap': [{'source': 'rule', 'raw': 'market capitalization is $12 billion', 'value': 12000000000.0, 'unit': 'billion', 'currency': 'USD'}], 'EPS': [{'source': 'rule', 'raw': 'EPS is $3.50', 'value': 3.5, 'unit': '', 'currency': 'USD'}], 'revenue_growth': [], 'stock_price_trend': [{'source': 'rule', 'raw': 'stock price rose 5%'}]}, 'events': {'IPO': [{'sentence': 'announced an IPO on June 5, 2024.', 'date': '2024-06-05T00:00:00'}], 'M&A': [], 'earnings_call': [], 'dividend': []}, 'verified': {'tickers': [{'ticker': 'T', 'price': 24.56, 'name': 'AT&T Inc.'}, {'ticker': 'AAPL', 'price': 270.14, 'name': 'Apple Inc.'}]}}
